In [114]:
import geopandas as gpd
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import altair as alt
alt.data_transformers.enable('json')


DataTransformerRegistry.enable('json')

In [115]:
seg = gpd.read_file('../data/pointsWithSeg.geojson')

In [117]:
seg

,fid,SEG_ID,pointId,wall,lives,building,infrastructure,road,sidewalk,sky,...,lives_q,building_q,infrastructure_q,road_q,sidewalk_q,sky_q,green_q,transportation_q,publicservice_q,geometry
0,1,420708,000001,0.030779,0.003141,0.163661,0.020548,0.118350,0.033204,0.291897,...,0.6,0.6,0.6,0.4,1.0,0.8,0.2,0.4,0.8,POINT (-75.16460 39.95994)
1,2,422065,000002,0.047397,0.008571,0.367944,0.009722,0.108147,0.035982,0.209341,...,1.0,1.0,0.2,0.4,1.0,0.6,0.0,0.0,0.4,POINT (-75.16357 39.96050)
2,3,420702,000003,0.034286,0.003337,0.178823,0.008427,0.237291,0.061395,0.024113,...,0.6,0.8,0.2,1.0,1.0,0.0,0.6,0.0,0.2,POINT (-75.16619 39.96013)
3,5,420696,000004,0.019361,0.001505,0.331703,0.004485,0.274849,0.073919,0.099125,...,0.4,1.0,0.0,1.0,1.0,0.2,0.0,0.6,0.6,POINT (-75.16776 39.96033)
4,8,422066,000005,0.051927,0.002919,0.139440,0.039439,0.035637,0.003948,0.092139,...,0.6,0.6,1.0,0.0,0.2,0.0,0.8,0.4,0.6,POINT (-75.16672 39.96089)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12312,24861,423467,012313,0.051546,0.011665,0.146067,0.052584,0.061044,0.014317,0.148602,...,1.0,0.6,1.0,0.2,0.4,0.4,0.6,0.8,0.2,POINT (-75.19385 39.95761)
12313,24863,522448,012314,0.036714,0.003632,0.197326,0.015623,0.019405,0.003073,0.189704,...,0.6,0.8,0.4,0.0,0.0,0.4,0.8,0.4,0.4,POINT (-75.15766 40.00221)
12314,24868,240821,012315,0.021332,0.000403,0.250894,0.013468,0.059427,0.008680,0.310452,...,0.2,1.0,0.4,0.2,0.4,1.0,0.2,0.2,1.0,POINT (-75.16788 39.90467)
12315,24869,240822,012316,0.018760,0.001339,0.290828,0.009906,0.066447,0.007145,0.309186,...,0.4,1.0,0.2,0.2,0.2,0.8,0.2,0.0,0.8,POINT (-75.16553 39.90436)


In [129]:
# Initialize the Kmeans object
kmeans = KMeans(n_clusters=4, random_state=42)

cols = ['wall', 'lives', 'building', 'infrastructure', 'road', 'sidewalk', 'sky', 'green', 'transportation', 'publicservice']

clusteringData = seg[cols].copy()

# Scale the data features we want
scaler = StandardScaler()
scaledClusteringData = scaler.fit_transform(clusteringData)

In [130]:
# Run the fit!
kmeans.fit(scaledClusteringData)

# Save the cluster labels
seg['label'] = kmeans.labels_

In [131]:
segSummary = seg.groupby('label')[cols].mean().reset_index()
segSummary = segSummary.melt(id_vars=['label'],var_name='cata')
segSummary = pd.DataFrame(segSummary)

In [132]:
segSummary.sort_values('value')

,label,variable,value
38,2,publicservice,0.000056
39,3,publicservice,0.000092
36,0,publicservice,0.000141
37,1,publicservice,0.000178
6,2,lives,0.001334
4,0,lives,0.002462
7,3,lives,0.003551
5,1,lives,0.007247
20,0,sidewalk,0.007644
15,3,infrastructure,0.012805


In [177]:
segSummary["q"] = segSummary.groupby("variable").value.rank(pct=True)
segSummary

,label,variable,value,q
0,0,wall,0.026113,0.50
1,1,wall,0.028116,0.75
2,2,wall,0.019327,0.25
3,3,wall,0.052916,1.00
4,0,lives,0.002462,0.50
5,1,lives,0.007247,1.00
6,2,lives,0.001334,0.25
7,3,lives,0.003551,0.75
8,0,building,0.191498,0.75
9,1,building,0.247785,1.00


In [250]:
alt.Chart(segSummary).mark_circle().encode(
    # row='variable:N',
    x='variable:N',
    color=alt.Color('value:Q', scale=alt.Scale(scheme="viridis")),
    size='value:Q',
    y='label:N',
    # row='label'
).properties(
    width=600,
    height=300
)

alt.Chart(...)

In [227]:
alt.Chart(segSummary).mark_bar().encode(
    row='label:N',
    color=alt.Color(
        'variable:N',
        scale=alt.Scale(scheme="tableau10")),
    y = alt.X('variable:N', sort='-x'),
    x=alt.X('value:Q'),
).properties(
    width=500,
    height=100
).interactive()

alt.Chart(...)

In [265]:
bar = alt.Chart(segSummary).mark_bar().encode(
    y=alt.Y('label:N'),
    color=alt.Color(
        'variable:N',
        scale=alt.Scale(scheme="tableau20")),
    x=alt.X('value:Q', stack="normalize"),
    tooltip=[
        alt.Tooltip('label:N', title='Clustering Label'),
        alt.Tooltip('variable:N', title='Catagory'),
        alt.Tooltip('value:Q', title='Percentage',format=',.2f'), 
    ]
).properties(
    width=800,
    height=300
)

bar

alt.Chart(...)

In [200]:
seg

,fid,SEG_ID,pointId,wall,lives,building,infrastructure,road,sidewalk,sky,...,building_q,infrastructure_q,road_q,sidewalk_q,sky_q,green_q,transportation_q,publicservice_q,geometry,label
0,1,420708,000001,0.030779,0.003141,0.163661,0.020548,0.118350,0.033204,0.291897,...,0.6,0.6,0.4,1.0,0.8,0.2,0.4,0.8,POINT (-75.16460 39.95994),3
1,2,422065,000002,0.047397,0.008571,0.367944,0.009722,0.108147,0.035982,0.209341,...,1.0,0.2,0.4,1.0,0.6,0.0,0.0,0.4,POINT (-75.16357 39.96050),1
2,3,420702,000003,0.034286,0.003337,0.178823,0.008427,0.237291,0.061395,0.024113,...,0.8,0.2,1.0,1.0,0.0,0.6,0.0,0.2,POINT (-75.16619 39.96013),1
3,5,420696,000004,0.019361,0.001505,0.331703,0.004485,0.274849,0.073919,0.099125,...,1.0,0.0,1.0,1.0,0.2,0.0,0.6,0.6,POINT (-75.16776 39.96033),1
4,8,422066,000005,0.051927,0.002919,0.139440,0.039439,0.035637,0.003948,0.092139,...,0.6,1.0,0.0,0.2,0.0,0.8,0.4,0.6,POINT (-75.16672 39.96089),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12312,24861,423467,012313,0.051546,0.011665,0.146067,0.052584,0.061044,0.014317,0.148602,...,0.6,1.0,0.2,0.4,0.4,0.6,0.8,0.2,POINT (-75.19385 39.95761),0
12313,24863,522448,012314,0.036714,0.003632,0.197326,0.015623,0.019405,0.003073,0.189704,...,0.8,0.4,0.0,0.0,0.4,0.8,0.4,0.4,POINT (-75.15766 40.00221),0
12314,24868,240821,012315,0.021332,0.000403,0.250894,0.013468,0.059427,0.008680,0.310452,...,1.0,0.4,0.2,0.4,1.0,0.2,0.2,1.0,POINT (-75.16788 39.90467),0
12315,24869,240822,012316,0.018760,0.001339,0.290828,0.009906,0.066447,0.007145,0.309186,...,1.0,0.2,0.2,0.2,0.8,0.2,0.0,0.8,POINT (-75.16553 39.90436),0


In [249]:
# seg["lat"], seg['lon'] = seg.geometry.x,seg.geometry.y 
# alt.Shape()
alt.Chart(seg).mark_geoshape(
    # color='black'
).encode(
    # shape=alt.Shape(field='geometry',type="geojson"),
    color='label:N',
    # facet=alt.Facet('label:N', columns=2),
    # x = "lat",
    # y = 'lon',
).properties(
    width=800,
    height=800
)

alt.Chart(...)

In [212]:
seg

,fid,SEG_ID,pointId,wall,lives,building,infrastructure,road,sidewalk,sky,...,building_q,infrastructure_q,road_q,sidewalk_q,sky_q,green_q,transportation_q,publicservice_q,geometry,label
0,1,420708,000001,0.030779,0.003141,0.163661,0.020548,0.118350,0.033204,0.291897,...,0.6,0.6,0.4,1.0,0.8,0.2,0.4,0.8,POINT (-75.16460 39.95994),3
1,2,422065,000002,0.047397,0.008571,0.367944,0.009722,0.108147,0.035982,0.209341,...,1.0,0.2,0.4,1.0,0.6,0.0,0.0,0.4,POINT (-75.16357 39.96050),1
2,3,420702,000003,0.034286,0.003337,0.178823,0.008427,0.237291,0.061395,0.024113,...,0.8,0.2,1.0,1.0,0.0,0.6,0.0,0.2,POINT (-75.16619 39.96013),1
3,5,420696,000004,0.019361,0.001505,0.331703,0.004485,0.274849,0.073919,0.099125,...,1.0,0.0,1.0,1.0,0.2,0.0,0.6,0.6,POINT (-75.16776 39.96033),1
4,8,422066,000005,0.051927,0.002919,0.139440,0.039439,0.035637,0.003948,0.092139,...,0.6,1.0,0.0,0.2,0.0,0.8,0.4,0.6,POINT (-75.16672 39.96089),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12312,24861,423467,012313,0.051546,0.011665,0.146067,0.052584,0.061044,0.014317,0.148602,...,0.6,1.0,0.2,0.4,0.4,0.6,0.8,0.2,POINT (-75.19385 39.95761),0
12313,24863,522448,012314,0.036714,0.003632,0.197326,0.015623,0.019405,0.003073,0.189704,...,0.8,0.4,0.0,0.0,0.4,0.8,0.4,0.4,POINT (-75.15766 40.00221),0
12314,24868,240821,012315,0.021332,0.000403,0.250894,0.013468,0.059427,0.008680,0.310452,...,1.0,0.4,0.2,0.4,1.0,0.2,0.2,1.0,POINT (-75.16788 39.90467),0
12315,24869,240822,012316,0.018760,0.001339,0.290828,0.009906,0.066447,0.007145,0.309186,...,1.0,0.2,0.2,0.2,0.8,0.2,0.0,0.8,POINT (-75.16553 39.90436),0
